# Xử lý dữ liệu số lượng thuốc từ file Excel

## I. Import thư viện và cấu hình file

In [14]:
import pandas as pd
import re

In [15]:
# Tên của file Excel cần đọc
file_name = 'test.xlsx'

# Chỉ định các cột cần đọc:
# - Cột 10: Phương pháp điều trị
# - Cột 11: Y, BS khám bệnh
require_cols = [10, 11]

# Chỉ định số hàng của hàng tiêu đề
# Tất cả các hàng trước hàng này sẽ bị bỏ qua
header_row = 6

## II. Xử lý dữ liệu

### Bước 1: Tải và làm sạch dữ liệu

In [16]:
# Đọc dữ liệu từ file Excel
df = pd.read_excel(file_name, usecols=require_cols, header=header_row)

# Loại bỏ các hàng bị thiếu giá trị
df = df.dropna()

print('Dữ liệu đã được tải và làm sạch thành công!\n')
print(df.head())

Dữ liệu đã được tải và làm sạch thành công!

                                Phương pháp điều trị     Y, BS khám bệnh
1  SPORAL  100mg SL: 1 SN: 7; LINEZIN  4mg/ml SL:...  TRỊNH THỊ MINH HẢI
2  SPORAL  100mg SL: 14 SN: 7; LINEZIN  4mg/ml SL...  TRỊNH THỊ MINH HẢI
3  POVIDINE   10%, 20ml SL: 1 SN: ; COTRIMOXAZOLE...         HOÀNG LƯƠNG
4  CIPROBAY 500mg SL: 14 SN: 7; ACEMUC  200mg SL:...         HOÀNG LƯƠNG
5  METRONIDAZOL  250mg SL: 14 SN: 7; AUGMENTIN  1...         HOÀNG LƯƠNG


### Bước 2: Trích xuất tên bác sĩ

In [17]:
# Trích xuất tên bác sĩ từ cột cuối cùng của DataFrame
doctor_names = df.iloc[:, -1]

# Loại bỏ các tên trùng lặp
unique_doctor_names = doctor_names.drop_duplicates()

# Chuyển đổi các tên bác sĩ thành dạng danh sách
doctor_names_list = unique_doctor_names.tolist()

print('Tên bác sĩ đã được trích xuất thành công!\n')

Tên bác sĩ đã được trích xuất thành công!



### Bước 3: Xử lý tên thuốc và số lượng thuốc cho từng bác sĩ 

In [18]:
# Tạo một từ điển (python dictionary) để lưu trữ tên thuốc và số lượng
medicine_dict = {}

# Hàm phân tích tên thuốc và số lượng từ một chuỗi
def parse_medicine(medicine_str):
    # Dùng biểu thức chính quy (regular expression) để trích xuất tên thuốc và số lượng
    # Ví dụ: 'SPORAL 100mg SL: 1 SN: 7' -> name: 'SPORAL 100mg', quantity: '1'
    match = re.match(r'(.+?) SL: (\d+)', medicine_str)

    if match:
        return match.groups()

    return None, None

# Lặp qua từng hàng trong DataFrame
# để xử lý tên thuốc và số lượng cho từng bác sĩ
for index, row in df.iterrows():
    # Lấy tên bác sĩ từ cột cuối cùng
    doctor_name = row.iloc[-1]

    # Lấy danh sách thuốc từ cột kế cuối và tách chúng bằng dấu ';'
    # Ví dụ: 'SPORAL 100mg SL: 1; SPORAL 200mg SL: 2' -> ['SPORAL 100mg SL: 1', 'SPORAL 200mg SL: 2']
    medicines = row.iloc[0].split(';')
    
    # Lặp qua từng thuốc trong danh sách thuốc
    for medicine in medicines:
        med_name, quantity = parse_medicine(medicine.strip())
        
        if med_name and quantity:
            if med_name not in medicine_dict:
                medicine_dict[med_name] = {doctor: 0 for doctor in doctor_names_list}

            medicine_dict[med_name][doctor_name] += int(quantity)

print('Tên thuốc và số lượng đã được xử lý thành công!\n')

print('Đây là 3 tên thuốc đầu tiên và số lượng của chúng được kê bởi mỗi bác sĩ:\n')
for med_name, quantities in list(medicine_dict.items())[:3]:
    print(f"Thuốc: {med_name}")

    for doctor, quantity in quantities.items():
        print(f"  - {doctor}: {quantity}")
print('\n')

Tên thuốc và số lượng đã được xử lý thành công!

Đây là 3 tên thuốc đầu tiên và số lượng của chúng được kê bởi mỗi bác sĩ:

Thuốc: SPORAL  100mg
  - TRỊNH THỊ MINH HẢI: 15
  - HOÀNG LƯƠNG: 0
  - TRƯƠNG THẾ ANH: 0
  - TRƯƠNG KIM TRI: 0
Thuốc: LINEZIN  4mg/ml
  - TRỊNH THỊ MINH HẢI: 42
  - HOÀNG LƯƠNG: 0
  - TRƯƠNG THẾ ANH: 0
  - TRƯƠNG KIM TRI: 0
Thuốc: ENTEROGERMINA  4 tỷ/5ml
  - TRỊNH THỊ MINH HẢI: 28
  - HOÀNG LƯƠNG: 0
  - TRƯƠNG THẾ ANH: 0
  - TRƯƠNG KIM TRI: 0




### Bước 4: Tính tổng số lượng của từng loại thuốc

In [19]:
# Tạo DataFrame từ từ điển thuốc
# với các tên bác sĩ là tiêu đề cột và tên thuốc là nhãn của các hàng
result_df = pd.DataFrame.from_dict(medicine_dict, orient='index', columns=doctor_names_list)

# Tạo cột mới 'TỔNG' chứa tổng số lượng thuốc của mỗi bác sĩ
result_df['>>> TỔNG <<<'] = result_df.sum(axis=1)

print('Tổng số lượng thuốc của mỗi bác sĩ đã được tính toán thành công!\n')

Tổng số lượng thuốc của mỗi bác sĩ đã được tính toán thành công!



### Bước 5: Ghi dữ liệu đã xử lý vào một sheet mới trong file Excel

In [20]:
print('Đang ghi kết quả vào file Excel...\n')

# Ghi kết quả vào sheet mới có tên 'Kết quả'
with pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    result_df.to_excel(writer, sheet_name='Kết quả', startrow=0)

print('>>> ĐÃ XONG! <<<')
print('Tên thuốc và số lượng thuốc của mỗi bác sĩ đã được ghi vào sheet "Kết quả".')

Đang ghi kết quả vào file Excel...

>>> ĐÃ XONG! <<<
Tên thuốc và số lượng thuốc của mỗi bác sĩ đã được ghi vào sheet "Kết quả".
